<font color="red"> WIP. More in-depth elaboration starting from Phase 1: Connect required</font>

# Running a Federated Cycle with Synergos

This tutorial aims to give you an understanding of how to use the synergos package to run a full federated learning cycle. 

In a federated learning system, there are many contributory participants, known as Worker nodes, which receive a global model to train on, with their own local dataset. The dataset does not leave the individual Worker nodes at any point, and remains private to the node.

The job to synchronize, orchestrate and initiate an federated learning cycle, falls on a Trusted Third Party (TTP). The TTP pushes out the global model architecture and parameters for the individual nodes to train on, calling upon the required data, based on tags, e.g "training", which points to relevant data on the individual nodes. At no point does the TTP receive, copy or access the Worker nodes' local datasets.

In this tutorial, you will go through the steps required by each participant (TTP and Worker), by simulating each of them locally with docker containers. Specifically, we will simulate a TTP and 2 Workers. 

At the end of this, we will have:
- Connected the participants
- Trained the model
- Evaluate the model

## Initiating the docker containers

Before we begin, we have to start the docker containers.

Firstly, follow the instructions on __\<path to pysyft_ttp readme\>__ and __\<path to pysyft_worker readme\>__ on how to build the required docker images for TTP and Workers.

Next, in <u>separate</u> CLI terminals, run the following command:

**Note: For Windows users, it is advisable to use powershell or command prompt based interfaces**

**Worker 1**

```docker run -v <directory heart_disease/data1>:/worker/data -v <directory heart_disease/outputs_1>:/worker/outputs --name worker_1 worker:pysyft_demo```

**Worker 2**

```docker run -v <directory heart_disease/data2>:/worker/data -v <directory heart_disease/outputs_2>:/worker/outputs --name worker_2 worker:pysyft_demo```

**TTP**

```docker run -p 0.0.0.0:5000:5000 -p 5678:5678 -p 8020:8020 -p 8080:8080 -v <directory /heart_disease/mlflow_test>:/ttp/mlflow --name ttp --link worker_1 --link worker_2 ttp:pysyft_demo```


Once ready, for each terminal, you should see that a Flask app is running on http://0.0.0.0:5000 of the container.

You are now ready for the next step.

## Configuration

In a new terminal, run `docker inspect bridge` and find the IPv4Address for each container. Ideally, the containers should have the following addresses:
- worker_1 address: 172.17.0.2
- worker_2 address: 172.17.0.3
- ttp address: 172.17.0.4

<font color="red">Not sure if need to run configure.py</font>

Run the following cells below.

**Note: For Windows users, `host` should be Docker Desktop VM's IP. Follow [this](https://stackoverflow.com/questions/58073936/how-to-get-ip-address-of-docker-desktop-vm) on instructions to find IP**

In [2]:
from synergos import Driver

host = "localhost" # Different for Windows users
port = 5000

# Initiate Driver
driver = Driver(host=host, port=port)

## Phase 1: Connect

Submitting TTP & Participant metadata

#### 1A. TTP controller creates a project

In [3]:
driver.projects.create(
    project_id="test_project",
    action="classify",
    incentives={
        'tier_1': [],
        'tier_2': [],
        'tier_3': []
    }
)

{'data': {'doc_id': '1',
  'kind': 'Project',
  'key': {'project_id': 'test_project'},
  'relations': {'Experiment': [],
   'Run': [],
   'Registration': [],
   'Tag': [],
   'Model': [],
   'Validation': [],
   'Prediction': []},
  'action': 'classify',
  'incentives': {'tier_3': [], 'tier_2': [], 'tier_1': []}},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'projects.post',
 'params': {}}

#### 1B. TTP controller creates an experiment

In [4]:
driver.experiments.create(
    project_id="test_project",
    expt_id="test_experiment",
    model = [
        # Input: N, C, Height, Width [N, 1, 28, 28]
        {
            "activation": "relu",
            "is_input": True,
            "l_type": "Conv2d",
            "structure": {
                "in_channels": 1, 
                "out_channels": 4, # [N, 4, 32, 32]
                "kernel_size": 3,
                "stride": 1,
                "padding": 1
            }
        },
        {
            "activation": None,
            "is_input": False,
            "l_type": "Flatten",
            "structure": {}
        },
        # ------------------------------
        {
            "activation": "sigmoid",
            "is_input": False,
            "l_type": "Linear",
            "structure": {
                "bias": True,
                "in_features": 4 * 32 * 32,
                "out_features": 1
            }
        }
    ]
)

{'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'experiments.post',
 'params': {'project_id': 'test_project'},
 'data': {'created_at': '2020-09-25 08:55:21 N',
  'key': {'expt_id': 'test_experiment', 'project_id': 'test_project'},
  'model': [{'activation': 'relu',
    'is_input': True,
    'l_type': 'Conv2d',
    'structure': {'in_channels': 1,
     'kernel_size': 3,
     'out_channels': 4,
     'padding': 1,
     'stride': 1}},
   {'activation': None,
    'is_input': False,
    'l_type': 'Flatten',
    'structure': {}},
   {'activation': 'sigmoid',
    'is_input': False,
    'l_type': 'Linear',
    'structure': {'bias': True, 'in_features': 4096, 'out_features': 1}}],
  'relations': {'Run': [], 'Model': [], 'Validation': [], 'Prediction': []},
  'doc_id': 1,
  'kind': 'Experiment'}}

#### 1C. TTP controller creates a run

In [5]:
driver.runs.create(
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run",
    rounds=2, 
    epochs=1,
    base_lr=0.0005,
    max_lr=0.005,
    criterion="NLLLoss"
)

{'data': {'doc_id': '1',
  'kind': 'Run',
  'key': {'project_id': 'test_project',
   'expt_id': 'test_experiment',
   'run_id': 'test_run'},
  'relations': {'Model': [], 'Validation': [], 'Prediction': []},
  'rounds': 2,
  'epochs': 1,
  'lr': 0.001,
  'lr_decay': 0.1,
  'weight_decay': 0.0,
  'seed': 42,
  'precision_fractional': 5,
  'mu': 0.1,
  'l1_lambda': 0.0,
  'l2_lambda': 0.0,
  'base_lr': 0.0005,
  'max_lr': 0.005,
  'patience': 10,
  'delta': 0.0},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'runs.post',
 'params': {'project_id': 'test_project', 'expt_id': 'test_experiment'}}

#### 1D. Participants registers their servers' configurations

In [6]:
driver.participants.create(
    participant_id="test_participant_1",
    host='172.17.0.2',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

driver.participants.create(
    participant_id="test_participant_2",
    host='172.17.0.3',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

{'data': {'doc_id': '2',
  'kind': 'Participant',
  'key': {'participant_id': 'test_participant_2'},
  'relations': {'Registration': [], 'Tag': []},
  'id': 'test_participant_2',
  'host': '172.17.0.3',
  'port': 8020,
  'log_msgs': True,
  'verbose': True,
  'f_port': 5000},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'participants.post',
 'params': {}}

#### 1E. Participants registers their role in a specific project

In [7]:
driver.registrations.create(
    project_id="test_project",
    participant_id="test_participant_1",
    role="guest"
)

driver.registrations.create(
    project_id="test_project",
    participant_id="test_participant_2",
    role="host"
)

{'data': {'doc_id': '2',
  'kind': 'Registration',
  'key': {'project_id': 'test_project',
   'participant_id': 'test_participant_2'},
  'relations': {'Tag': []},
  'project': {'incentives': {'tier_3': [], 'tier_2': [], 'tier_1': []},
   'start_at': None},
  'participant': {'id': 'test_participant_2',
   'host': '172.17.0.3',
   'port': 8020,
   'f_port': 5000,
   'log_msgs': True,
   'verbose': True},
  'role': 'host'},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'registration.post',
 'params': {}}

#### 1F. Participants registers their tags for a specific project

In [8]:
driver.tags.create(
    project_id="test_project",
    participant_id="test_participant_1",
    train=[["train"]],
    evaluate=[["evaluate"]],
    predict = [["predict"]]
)

driver.tags.create(
    project_id="test_project",
    participant_id="test_participant_2",
    train=[["train"]],
    evaluate=[["evaluate"]],
    predict=[["predict"]]
)

{'data': {'doc_id': '2',
  'kind': 'Tag',
  'key': {'project_id': 'test_project',
   'participant_id': 'test_participant_2'},
  'train': [['train']],
  'evaluate': [['evaluate']],
  'predict': [['predict']]},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'tags.post',
 'params': {'project_id': 'test_project',
  'participant_id': 'test_participant_2'}}

## Phase 2:
Alignment, Training & Optimisation

#### 2A. Perform multiple feature alignment to dynamically configure datasets and models for cross-grid compatibility

In [9]:
driver.alignments.create(project_id="test_project")

{'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'alignments.post',
 'params': {'project_id': 'test_project'},
 'data': [{'created_at': '2020-09-25 08:56:35 N',
   'evaluate': {'X': [], 'y': []},
   'key': {'participant_id': 'test_participant_1',
    'project_id': 'test_project'},
   'link': {'alignment_id': '042790a0ff0d11eaaa510242ac110004',
    'registration_id': 'd896346eff0c11eaa6770242ac110004',
    'tag_id': 'd8f4c8b2ff0c11ea922e0242ac110004'},
   'predict': {'X': [], 'y': []},
   'train': {'X': [], 'y': []},
   'relations': {},
   'doc_id': 1,
   'kind': 'Alignment'},
  {'created_at': '2020-09-25 08:56:35 N',
   'evaluate': {'X': [], 'y': []},
   'key': {'participant_id': 'test_participant_2',
    'project_id': 'test_project'},
   'link': {'alignment_id': '042a4ad4ff0d11eaaa510242ac110004',
    'registration_id': 'd89a4144ff0c11ea9d280242ac110004',
    'tag_id': 'd8f7fa14ff0c11eab5470242ac110004'},
   'predict': {'X': [], 'y': []},
   'train': {'X': [], 'y': []

#### 2B. Trigger training across the federated grid

In [10]:
model_resp = driver.models.create(
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run"
)

#### 2C. Trigger hyperparameter optimisation across the grid

## Phase 3: EVALUATE 
Validation & Predictions

#### 3A. Perform validation(s) of combination(s)

In [10]:
driver.validations.create(
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run"
)

{'message': 'Internal Server Error'}

#### 3B. Perform prediction(s) of combination(s)

In [ ]:
driver.predictions.create(
    tags={"test_project": [["iid_1"]]},
    participant_id="test_participant_1",
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run"
)